In [1]:
# mamba create --name python-3.11.4 python=3.11.4

In [2]:
### security parameters
RSA_KEY_SIZE = 3072  # RSA key size for 128 bits of security (modulu size)
RSA_PRIME_SIZE = int(RSA_KEY_SIZE / 2)
ACCUMULATED_PRIME_SIZE = 128  # taken from: LLX, "Universal accumulators with efficient nonmembership proofs", construction 1

GENERATED_RESULTS = "GENERATED_RESULTS"
KHADER_SETUP_PARAMS_FILE = "KHADER_setup_params.csv"
KHADER_SIMULATIONS_RESULTS_FILE = "KHADER_simulation_results.csv"

PoEDPP_SETUP_PARAMS_FILE = "PoEDPP_setup_params.csv"
PoEDPP_SIMULATIONS_RESULTS_FILE = "PoEDPP_simulation_results.csv"

BLOCK_SIZE_BYTES = 384
FILE_SIZES_MB =  [i for i in range(10000,110000,10000)]
# print(FILE_SIZES_MB)
# print(len(FILE_SIZES_MB))

In [3]:
#%pip install pycryptodomex
#%pip install pandas
import random
import hashlib
import secrets
import math
from Cryptodome.Random import get_random_bytes
from Cryptodome.Cipher import AES
from base64 import b64decode
import binascii
import random
import time
import csv
import pandas as pd

def rabin_miller(num):
    # Returns True if num is a prime number.

    s = num - 1
    t = 0
    while s % 2 == 0:
        # keep halving s while it is even (and use t
        # to count how many times we halve s)
        s = s // 2
        t += 1

    for trials in range(5): # try to falsify num's primality 5 times
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # this test does not apply if v is 1.
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True


def is_prime(num):
    # Return True if num is a prime number. This function does a quicker
    # prime number check before calling rabin_miller().

    if (num < 2):
        return False # 0, 1, and negative numbers are not prime

    # About 1/3 of the time we can quickly determine if num is not prime
    # by dividing by the first few dozen prime numbers. This is quicker
    # than rabin_miller(), but unlike rabin_miller() is not guaranteed to
    # prove that a number is prime.
    lowPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]

    if num in lowPrimes:
        return True

    # See if any of the low prime numbers can divide num
    for prime in lowPrimes:
        if (num % prime == 0):
            return False

    # If all else fails, call rabin_miller() to determine if num is a prime.
    return rabin_miller(num)


# ### key generation

# ### security parameters
# RSA_KEY_SIZE = 3072  # RSA key size for 128 bits of security (modulu size)
# RSA_PRIME_SIZE = int(RSA_KEY_SIZE / 2)

def generate_large_prime(RSA_PRIME_SIZE):
  while True:
    num = secrets.randbelow(pow(2,RSA_PRIME_SIZE))
    if is_prime(num):
      return num

def generate_two_primes(RSA_PRIME_SIZE):
  p = generate_large_prime(RSA_PRIME_SIZE)
  while True:
    q = secrets.randbelow(pow(2,RSA_PRIME_SIZE))
    if p != q :
     return p,q


######### concat_bytes
def concat_bytes(*args):
    concatenated_bytes = b""
    for arg in args:
        if isinstance(arg, bytes):
            concatenated_bytes += arg
        else:
            try:
                # If the argument is not bytes, try converting it to bytes
                concatenated_bytes += bytes(arg, 'utf-8')
            except TypeError:
                raise TypeError("All arguments must be bytes or convertible to bytes.")

    return concatenated_bytes


# generate files of different sizes
# usage
# file_sizes_mb = [128, 256, 384, 512, 640, 768, 896, 1024, 1152]
# for size_mb in file_sizes_mb:
#     generate_file(size_mb)

def generate_file(FILE_SIZES_MB, BLOCK_SIZE_BYTES):
    data = "أعوذ بالله بالله بالله من الشيطان الرجيم، ، ،أعوذ بالله من الشيطان الرجيم بسم الله الرحمن الرحيم قُلْ هُوَ اللَّهُ أَحَدٌ (1) اللَّهُ الصَّمَدُ (2) لَمْ يَلِدْ وَلَمْ يُولَدْ (3) وَلَمْ يَكُنْ لَهُ كُفُوًا أَحَدٌ(4)"
    block = data.encode('utf-8')
    file_size_bytes = FILE_SIZES_MB * 1024 * 1024
    # file_size_bytes = FILE_SIZES_MB  # to be modify 
    with open(GENERATED_RESULTS+"/"+f"output_{FILE_SIZES_MB}MB.txt", "wb") as file:
        bytes_written = 0
        while bytes_written < file_size_bytes:
            bytes_to_write = min(BLOCK_SIZE_BYTES, file_size_bytes - bytes_written)
            file.write(block[:bytes_to_write])
            bytes_written += bytes_to_write

# Example usage to read and output the entire file or the first 10 blocks
# file_path = "output_128MB.bin"
# # read_and_output_blocks(file_path)  # Read and output the entire file
# read_and_output_blocks(file_path, num_blocks=10)  # Read and output the first 10 blocks
def read_and_output_blocks(file_path, BLOCK_SIZE_BYTES, num_blocks=None):
    with open(GENERATED_RESULTS+"/"+file_path, "rb") as file:
        blocks_read = 0
        while num_blocks is None or blocks_read < num_blocks:
            block = file.read(BLOCK_SIZE_BYTES)
            if not block:
                break
            yield block
            blocks_read += 1

######### encrypts
def encrypt(plaintext, key, mode):
  enc_obj = AES.new(key, mode)
  cipher_text,auth_tag=enc_obj.encrypt_and_digest(plaintext)
  return(cipher_text, auth_tag, enc_obj.nonce)

######### decrypts
def decrypt(ciphertext, key, mode):
  (ciphertext,  authTag, nonce) = ciphertext
  encobj = AES.new(key,  mode, nonce)
  return(encobj.decrypt_and_verify(ciphertext, authTag))

######### left_shift_bytes  bi_1 << 1
def left_shift_bytes(bytes_data, shift_bits=1):
    if not isinstance(bytes_data, bytes):
        raise TypeError("bytes_data must be of type 'bytes'.")

    if not isinstance(shift_bits, int) or shift_bits < 0:
        raise ValueError("Shift_bits must be a non-negative integer.")

    # Convert bytes to integer, perform left shift, and convert back to bytes
    shifted_int = int.from_bytes(bytes_data, 'big') << shift_bits
    shifted_bytes = shifted_int.to_bytes((shifted_int.bit_length() + 7) // 8, 'big')

    return shifted_bytes

def right_shift_bytes(bytes_data, shift_bits=1):
    if not isinstance(bytes_data, bytes):
        raise TypeError("bytes_data must be of type 'bytes'.")

    if not isinstance(shift_bits, int) or shift_bits < 0:
        raise ValueError("Shift_bits must be a non-negative integer.")

    # Convert bytes to integer, perform right shift, and convert back to bytes
    shifted_int = int.from_bytes(bytes_data, 'big') >> shift_bits
    shifted_bytes = shifted_int.to_bytes((shifted_int.bit_length() + 7) // 8, 'big')

    return shifted_bytes



def setup(RSA_PRIME_SIZE):
    # print(f'#############################')
    # print(f'#### Setup  ###########')
    # print(f'#############################')
    p,q = generate_two_primes(RSA_PRIME_SIZE)
    N = p*q
    phi_n = (p-1)*(q-1)
    K_H = get_random_bytes(16) # 128-bit authentication code
    K_E = get_random_bytes(32) # 128 bits AES
    Acc = secrets.randbelow(N) # A0 = g
    # TRT = []
    # ct_table = []

    return N, phi_n, K_H, K_E, Acc #, TRT, ct_table


def add_khader(A, TRT, ct_table, ciphertext, n, phi_n):
    # print(f'#############################')
    # print(f'#### Add element ###########')
    # print(f'#############################')

    bi_1 = concat_bytes(ciphertext[0], ciphertext[1], ciphertext[2])
    # add tag to TRT
    TRT.append(ciphertext[1])

    # left_shift
    bi_2 = left_shift_bytes(bi_1)
    ct_table.append(bi_2)
    return pow(A, int(binascii.hexlify(bi_2),16), phi_n) % n

def add_khader_2(A, TRT, ct_table, ciphertext, n, phi_n):
    # print(f'#############################')
    # print(f'#### Add element ###########')
    # print(f'#############################')

    bi_1 = concat_bytes(ciphertext[0], ciphertext[1], ciphertext[2])
    # add tag to TRT
    TRT.append(ciphertext[1])

    # left_shift
    bi_2 = left_shift_bytes(bi_1)
    ct_table.append(bi_2)
    return pow(A, int(binascii.hexlify(bi_2),16), phi_n) % n



# get the index j
# compute w_j = g^{\prod_{0}^{n} i/=j  (bi_2) mod phi_n } mod N
def prove_membership_khader(A0, TRT, ct_table, j, n, phi_n):
    # get the j-th element
    # print(f'#############################')
    # print(f'Prove membership ')
    # print(f'#############################')
    product = 1
    for element in ct_table:
        if  int(binascii.hexlify(element),16) != int(binascii.hexlify(ct_table[j]),16):
            product *= int(binascii.hexlify(element),16) % phi_n
        # else:
        #   print(f'#### exist = ct_table[{j}] = {ct_table[j]}')

    A = pow(A0, product , phi_n) % n
    return A, ct_table[j]

def verify_membership_khader(Accb, wj, bj_2, n, phi_n, K_E):
    # print(f'#############################')
    # print(f'verify membership ')
    # print(f'#############################')
    # print(f'pow(wj, cj, phi_n) % n = {pow(wj, int(binascii.hexlify(bj_2),16) % phi_n, phi_n) % n} ')
    # print(f'Accb = {Accb}')

    # print(f'verify tag ')
    # ciphertext to tuple

    # ###### decryption phase
    bi_1_dec = right_shift_bytes(bj_2)
    # ### nonce = extract the last 16 bytes
    nonce = bi_1_dec[-16:]
    tag = bi_1_dec[-32:-16]
    cipher_text_csp = bi_1_dec[0:-32]
    # print(f'cipher_text_csp = {binascii.hexlify(cipher_text_csp).decode()} ')
    # print(f'tag = {binascii.hexlify(tag).decode()} ')
    # print(f'nonce = {binascii.hexlify(nonce).decode()} ')
    enc_data_to_csp = (cipher_text_csp, tag, nonce)
    ### construct the ciphertext from bi_1_dec
    try:
        res = decrypt(enc_data_to_csp, K_E, AES.MODE_GCM)
        # print ("\n\nDecrypted:\t",res.decode())
    except TypeError:
        raise TypeError("decryption failed")

    # BlockGen()

    return pow(wj, int(binascii.hexlify(bj_2),16), phi_n) % n == Accb

def lookup(file_path, BLOCK_SIZE_BYTES, index):
    try:
        with open(GENERATED_RESULTS+"/"+file_path, "rb") as file:
            # Calculate the starting byte position of the block
            start_byte = index * BLOCK_SIZE_BYTES

            # Move to the starting byte position in the file
            file.seek(start_byte)

            # Read the block of the specified size
            block = file.read(BLOCK_SIZE_BYTES)
            return block

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except IOError:
        print(f"Error: An I/O error occurred while reading the file.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

def save_setup_paramters(setup_params, output_csv_file):
    
    try:
        with open(GENERATED_RESULTS+"/"+ output_csv_file, 'w', newline='') as paramfile:
            fieldnames = ['N', 'phi_N', 'K_H', 'K_E', 'Acc0', 'acct' ]
            writer = csv.DictWriter(paramfile, fieldnames=fieldnames)
            writer.writeheader()
            for param in setup_params:
                writer.writerow({'N':param[0], 'phi_N':param[1], 'K_H':param[2], 'K_E':param[3], 'Acc0':param[4], 'acct': param[5]})
    except IOError as e:
        print(f"Error: {e}")

def save_simulation_results(simulation_results, output_csv_file):
  # save info to csv file_size | # blocks | size of enc block  | setup_params_size | Block_Gen_time | proof_generation_time 
  # Simulation_results.append((size,index+1, enc_block_size, tag_size, len(setup_params_bytes), block_gen_time , proof_generation_time , proof_verification_time ))
    try:
        with open(GENERATED_RESULTS+"/"+ output_csv_file, 'w', newline='') as simfile:
            fieldnames = ['file-size', 'num of blocks', 'enc block size','tag size' ,'setup params size', 'blockGen time', 'proofGen time', 'proofVer time' ]
            writer = csv.DictWriter(simfile, fieldnames=fieldnames)
            writer.writeheader()
            for param in simulation_results:
                writer.writerow({'file-size':param[0], 'num of blocks':param[1], 'enc block size':param[2],'tag size':param[3],  'setup params size':param[4], 'blockGen time':param[5], 'proofGen time': param[6], 'proofVer time': param[7]})
    except IOError as e:
        print(f"Error: {e}")

def int_to_bytes(N):
    num_bits = N.bit_length()
    num_bytes = (num_bits + 7) // 8
    return N.to_bytes(num_bytes, 'big')

In [4]:
# # # Generate test files of [128, 256, 384, 512, 640, 768, 896, 1024, 1152]
# # # Generate files of MB
# # for size in FILE_SIZES_MB:
# #     generate_file(size,BLOCK_SIZE_BYTES)


# # # Generate files of 10 ...10000 blocks of 384
# data = "أعوذ بالله بالله بالله من الشيطان الرجيم، ، ،أعوذ بالله من الشيطان الرجيم بسم الله الرحمن الرحيم قُلْ هُوَ اللَّهُ أَحَدٌ (1) اللَّهُ الصَّمَدُ (2) لَمْ يَلِدْ وَلَمْ يُولَدْ (3) وَلَمْ يَكُنْ لَهُ كُفُوًا أَحَدٌ(4)"
# # FILE_SIZES_MB = [10, 50, 100, 500, 1000, 5000, 10000]
# # FILE_SIZES_MB = [1020]
# for size in FILE_SIZES_MB:
#   print(f'create file of {size}')
#   data_list = [data for _ in range(size)]
#   file_name = f"output_{size}.txt"
#   with open(GENERATED_RESULTS+"/"+file_name, "wb") as file:
#     for data in data_list:
#             file.write(data.encode('utf-8'))
#             # print(len(bytes(data.encode('utf-8'))))

# PoEDDP simulation

In [5]:

def generate_large_prime(num_of_bits):
    while True:
        num = secrets.randbelow(pow(2, num_of_bits))
        if is_prime(num):
            return num


def generate_two_large_distinct_primes(num_of_bits):
    p = generate_large_prime(num_of_bits)
    while True:
        q = generate_large_prime(num_of_bits)
        while q != p:
            return p, q


def hash_to_prime(x, num_of_bits=128, nonce=0):
    while True:
        num = hash_to_length(x + nonce, num_of_bits)
        if is_prime(num):
            return num, nonce
        nonce = nonce + 1


def hash_to_length(x, num_of_bits):
    pseudo_random_hex_string = ""
    num_of_blocks = math.ceil(num_of_bits / 256)
    for i in range(0, num_of_blocks):
        pseudo_random_hex_string += hashlib.sha256(str(x + i).encode()).hexdigest()

    if num_of_bits % 256 > 0:
        pseudo_random_hex_string = pseudo_random_hex_string[int((num_of_bits % 256)/4):]  # we do assume divisible by 4
    return int(pseudo_random_hex_string, 16)



def concat(*arg):
    res = ""
    for i in range(len(arg)):
        res += str(arg[i])
    return int(res)

# used in prove non-memebership
def bezoute_coefficients(a, b):
    o = xgcd(a, b)
    return o[1], o[2]

# used in prove non-membership
def mul_inv(b, n):
    g, x, _ = xgcd(b, n)
    if g == 1:
        return x % n
# used in prove non-memebership
def xgcd(b, a):
    x0, x1, y0, y1 = 1, 0, 0, 1
    while a != 0:
        q, b, a = b // a, a, b % a
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return b, x0, y0

# This is the fastest method available
# used in aggregation: computes primes products 
def calculate_product(lst):
    r = 1
    for x in lst:
        r *= x
    return r

######### encrypts
def encrypt(plaintext, key, mode):
  enc_obj = AES.new(key, mode)
  cipher_text,auth_tag=enc_obj.encrypt_and_digest(plaintext)
  return(cipher_text, auth_tag, enc_obj.nonce)

######### decrypts
def decrypt(ciphertext, key, mode):
  (ciphertext,  authTag, nonce) = ciphertext
  encobj = AES.new(key,  mode, nonce)
  return(encobj.decrypt_and_verify(ciphertext, authTag))

def lookup(file_path, BLOCK_SIZE_BYTES, index):
    try:
        with open(GENERATED_RESULTS+"/"+file_path, "rb") as file:
            # Calculate the starting byte position of the block
            start_byte = index * BLOCK_SIZE_BYTES

            # Move to the starting byte position in the file
            file.seek(start_byte)

            # Read the block of the specified size
            block = file.read(BLOCK_SIZE_BYTES)
            return block

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except IOError:
        print(f"Error: An I/O error occurred while reading the file.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None
def int_to_bytes(N):
    num_bits = N.bit_length()
    num_bytes = (num_bits + 7) // 8
    return N.to_bytes(num_bytes, 'big')


def setup_podepp(RSA_PRIME_SIZE):
    p, q = generate_two_large_distinct_primes(RSA_PRIME_SIZE)
    N = p*q
    A0 = secrets.randbelow(N)
    K_E = get_random_bytes(32) # 128 bits AES
    return N, A0, K_E, dict()


def add_podepp(A, S, x, n):
    hash_prime, nonce = hash_to_prime(x, ACCUMULATED_PRIME_SIZE)
    if hash_prime in S.keys():
        # print(f'{hash_prime} exist : must not change')
        # print(f'before A = {A}')
        return A
    else:
        # print(f'{hash_prime} not exist : must change')
        # print(f'before A = {A}')
        S[hash_prime] = nonce
        A = pow(A, hash_prime, n)
        return A    
def prove_membership_podepp(A0, S, x, n):
    r = hash_to_prime(x, ACCUMULATED_PRIME_SIZE)[0]
    if r not in S.keys():
        return None
    else:
        product = 1
        for element in S.keys():
            if element != r:
                product *= element
        A = pow(A0, product, n)
        return A

def verify_membership_podepp(A, x, nonce, proof, n):
    hs = hash_to_prime(x=x, num_of_bits=ACCUMULATED_PRIME_SIZE, nonce=nonce)[0]
    return __verify_membership_podepp(A, hs , proof, n)

def __verify_membership_podepp(A, x, proof, n):
    return pow(proof, x, n) == A


def prove_membership_with_NIPoE_podepp(g, S, x, n, w):
    u = prove_membership_podepp(g, S, x, n)
    x_prime, x_nonce = hash_to_prime(x)
    (Q, l_nonce) = prove_exponentiation_podepp(u, x_prime, w, n)
    return Q, l_nonce, u

def prove_exponentiation_podepp(u, x, w, n):
    l, nonce = hash_to_prime(concat(x, u, w))  # Fiat-Shamir instead of interactive challenge
    q = x // l
    Q = pow(u, q, n)
    return Q, nonce

def verify_exponentiation(Q, l_nonce, u, x, x_nonce, w, n):

    x = hash_to_prime(x=x, nonce=x_nonce)[0]
    # print(f'x = {x}')
    return __verify_exponentiation(Q, l_nonce, u, x, w, n)

# helper function, does not do hash_to_prime on x
def __verify_exponentiation(Q, l_nonce, u, x, w, n):
    l = hash_to_prime(x=(concat(x, u, w)), nonce=l_nonce)[0]
    r = x % l
    # check (Q^l)(u^r) == w
    return (pow(Q, l, n) % n) * (pow(u, r, n) % n) % n == w  


######### concat_bytes
def concat_bytes(*args):
    concatenated_bytes = b""
    for arg in args:
        if isinstance(arg, bytes):
            concatenated_bytes += arg
        else:
            try:
                # If the argument is not bytes, try converting it to bytes
                concatenated_bytes += bytes(arg, 'utf-8')
            except TypeError:
                raise TypeError("All arguments must be bytes or convertible to bytes.")

    return concatenated_bytes

def read_and_output_blocks(file_path, BLOCK_SIZE_BYTES, num_blocks=None):
    with open(GENERATED_RESULTS+"/"+file_path, "rb") as file:
        blocks_read = 0
        while num_blocks is None or blocks_read < num_blocks:
            block = file.read(BLOCK_SIZE_BYTES)
            if not block:
                break
            yield block
            blocks_read += 1


def save_setup_paramters(setup_params, output_csv_file):
    
    try:
        with open(GENERATED_RESULTS+"/"+ output_csv_file, 'w', newline='') as paramfile:
            fieldnames = ['N', 'K_E', 'Acc0', 'acct' ]
            writer = csv.DictWriter(paramfile, fieldnames=fieldnames)
            writer.writeheader()
            for param in setup_params:
                writer.writerow({'N':param[0], 'K_E':param[1], 'Acc0':param[2], 'acct':param[3]})
    except IOError as e:
        print(f"Error: {e}")

def save_simulation_results(simulation_results, output_csv_file):
  # save info to csv file_size | # blocks | size of enc block  | setup_params_size | Block_Gen_time | proof_generation_time 
  # Simulation_results.append((size,index+1, enc_block_size, tag_size, len(setup_params_bytes), block_gen_time , proof_generation_time , proof_verification_time ))
    try:
        with open(GENERATED_RESULTS+"/"+ output_csv_file, 'w', newline='') as simfile:
            fieldnames = ['file-size', 'num of blocks', 'enc block size','tag size' ,'setup params size', 'blockGen time', 'proofGen time', 'proofVer time' ]
            writer = csv.DictWriter(simfile, fieldnames=fieldnames)
            writer.writeheader()
            for param in simulation_results:
                writer.writerow({'file-size':param[0], 'num of blocks':param[1], 'enc block size':param[2],'tag size':param[3],  'setup params size':param[4], 'blockGen time':param[5], 'proofGen time': param[6], 'proofVer time': param[7]})
    except IOError as e:
        print(f"Error: {e}")

In [6]:
print(f'start PoEDDP Scheme ...')
Simulation_results = []
hash_prime_bytes = 0

for size in FILE_SIZES_MB:
  print(f'On file {size}')
  file_name = f"output_{size}.txt"
  block_gen_time = any
  proof_generation_time = any
  proof_verification_time = any
  enc_block_size = 0
  tag_size = 0
  tik = 0
  # Setup and BlockGen
  setup_pp = []
  N, Acc0, K_E, HPM = setup_podepp(RSA_PRIME_SIZE)
  acct = Acc0
  product = 1
  
  with open(GENERATED_RESULTS+"/"+f"{file_name}_poedpp.enc", "wb") as enc_file:
    with open(GENERATED_RESULTS+"/"+f"{file_name}_poedpp.tag", "wb") as tag_file:
      for index, block in enumerate(read_and_output_blocks(file_name, 384, num_blocks=None)):
        # if index == 1:
          # BlockGen
          tik = time.time()
          ciphertext = encrypt(block, K_E, AES.MODE_GCM)
          # \tau <- H_prime(ci)
          c = concat_bytes(ciphertext[0], ciphertext[1], ciphertext[2])
          
          hash_prime, prime_nonce = hash_to_prime(int(binascii.hexlify(c),16), ACCUMULATED_PRIME_SIZE)
          HPM[hash_prime] = prime_nonce
          product *= hash_prime
          
          #get sizes for results
          enc_block_size = len(c)
          hash_prime_bytes = hash_prime.to_bytes((hash_prime.bit_length() + 7) // 8, 'big')
          tag_size = len(hash_prime_bytes)
          
          # store tag in file and enc 
          enc_file.write(c)
          tag_file.write(hash_prime_bytes)
          
          

  acct = pow(Acc0, product, N)
  setup_pp.append((N, K_E, Acc0, acct))
  setup_params_bytes = concat_bytes(int_to_bytes(N), K_E, int_to_bytes(Acc0),int_to_bytes(acct))
  tok = time.time()
  block_gen_time = tok-tik
  
  # chellenge index+1 - 1 = len(HPM) - 1 
  j = random.randint(0, index)

  
  # print(f'#############################')
  # print(f'PROOF GENERATION')
  # print(f'#############################')
  tik = time.time()
  
  # print(f'challenge')
  c_j = lookup(f"{file_name}_poedpp.enc", 416, 1)
  # print(block_j)
  nonce = c_j[-16:]
  tag_csp = c_j[-32:-16]
  cipher_text_csp = c_j[0:-32]

  enc_data_to_csp = (cipher_text_csp, tag_csp, nonce)
  tag_client = lookup(f"{file_name}_poedpp.tag", 16, 1)
  #init product 
  product = 1

  Q, l_nonce, u = prove_membership_with_NIPoE_podepp(Acc0, HPM, int(binascii.hexlify(c_j),16) , N, acct)
  tok=time.time()
  proof_generation_time = tok-tik
  
  # print(f'#############################')
  # print(f'verify membership ')
  # print(f'#############################')
  tik = time.time()
  x = int(binascii.hexlify(c_j),16)
  x_prim, x_nonce_prim  = hash_to_prime(x, ACCUMULATED_PRIME_SIZE)
  prime_nonce = HPM[x_prim]
  is_valid = verify_exponentiation(Q, l_nonce, u, x, prime_nonce , acct, N)
  
  print(f'proof is valid = {is_valid} tag is valid {tag_csp == tag_client}')
  try:
      res = decrypt(enc_data_to_csp, K_E, AES.MODE_GCM)
      # proof_exists = (pow(wj, int(binascii.hexlify(bi_1_dec),16), phi_n) % N == acct and tag_j == tag)
      print (f"proof exists = {is_valid} correct decryption = {res.decode()}")
  except TypeError:
      raise TypeError("decryption failed")
  tok = time.time()
  proof_verification_time = tok-tik
  # # print(f'file size = {size} encrypt block[{index+1}] of size= {len(bytes(block))}')
  # # save info to csv file_size | # blocks | size of enc block  | setup_params_size | Block_Gen_time | proof_generation_time 
  Simulation_results.append((size,index+1, enc_block_size, tag_size, len(setup_params_bytes), block_gen_time , proof_generation_time , proof_verification_time ))


# save setup parameters to csv 
  save_setup_paramters(setup_pp, PoEDPP_SETUP_PARAMS_FILE)

# Simulation_results to csv 
  save_simulation_results(Simulation_results, PoEDPP_SIMULATIONS_RESULTS_FILE)


start PoEDDP Scheme ...
On file 10000
proof is valid = True tag is valid False
proof exists = True correct decryption = أعوذ بالله بالله بالله من الشيطان الرجيم، ، ،أعوذ بالله من الشيطان الرجيم بسم الله الرحمن الرحيم قُلْ هُوَ اللَّهُ أَحَدٌ (1) اللَّهُ الصَّمَدُ (2) لَمْ يَلِدْ وَلَمْ يُولَدْ (3) وَلَمْ يَكُنْ لَهُ كُفُوًا أَحَدٌ(4)
On file 20000
proof is valid = True tag is valid False
proof exists = True correct decryption = أعوذ بالله بالله بالله من الشيطان الرجيم، ، ،أعوذ بالله من الشيطان الرجيم بسم الله الرحمن الرحيم قُلْ هُوَ اللَّهُ أَحَدٌ (1) اللَّهُ الصَّمَدُ (2) لَمْ يَلِدْ وَلَمْ يُولَدْ (3) وَلَمْ يَكُنْ لَهُ كُفُوًا أَحَدٌ(4)
On file 30000
proof is valid = True tag is valid False
proof exists = True correct decryption = أعوذ بالله بالله بالله من الشيطان الرجيم، ، ،أعوذ بالله من الشيطان الرجيم بسم الله الرحمن الرحيم قُلْ هُوَ اللَّهُ أَحَدٌ (1) اللَّهُ الصَّمَدُ (2) لَمْ يَلِدْ وَلَمْ يُولَدْ (3) وَلَمْ يَكُنْ لَهُ كُفُوًا أَحَدٌ(4)
On file 40000
proof is valid = True tag 